https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py

# Basic pytorch

In [2]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms


# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 


In [8]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


tensor(2.)
tensor(1.)
tensor(1.)


In [10]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

for i in range(100):
    # Forward pass.
    pred = linear(x)

    # Compute loss.
    loss = criterion(pred, y)
    print(i, 'loss: ', loss.item())

    # Backward pass.
    loss.backward()

    # Print out the gradients.
    #print ('dL/dw: ', linear.weight.grad) 
    #print ('dL/db: ', linear.bias.grad)

    # 1-step gradient descent.
    optimizer.step()
    optimizer.zero_grad()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[ 0.3279, -0.4911, -0.4888],
        [-0.1475,  0.4737, -0.1042]], requires_grad=True)
b:  Parameter containing:
tensor([-0.2334, -0.3609], requires_grad=True)
0 loss:  1.242653489112854
1 loss:  1.2276136875152588
2 loss:  1.21290123462677
3 loss:  1.1985089778900146
4 loss:  1.1844292879104614
5 loss:  1.170655369758606
6 loss:  1.1571801900863647
7 loss:  1.1439969539642334
8 loss:  1.131098985671997
9 loss:  1.1184799671173096
10 loss:  1.1061334609985352
11 loss:  1.0940535068511963
12 loss:  1.0822337865829468
13 loss:  1.0706686973571777
14 loss:  1.0593522787094116
15 loss:  1.048279047012329
16 loss:  1.0374435186386108
17 loss:  1.0268402099609375
18 loss:  1.016464114189148
19 loss:  1.0063098669052124
20 loss:  0.9963725805282593
21 loss:  0.9866472482681274
22 loss:  0.9771292805671692
23 loss:  0.9678139686584473
24 loss:  0.958696722984314
25 loss:  0.9497731328010559
26 loss:  0.9410387873649597
27 loss:  0.9324894547462463
28 loss:  0.

In [11]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
#y = torch.tensor(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [15]:
torch.cuda.is_available()

True

In [20]:
x = np.random.randn(100000)
y = np.random.randn(100000)
%timeit x.dot(y)

63.3 µs ± 969 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [24]:
x = torch.tensor(np.random.randn(100000)).to('cpu')
y = torch.tensor(np.random.randn(100000)).to('cpu')
%timeit x.dot(y)

17.5 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
x = torch.tensor(np.random.randn(100000)).to('cuda')
y = torch.tensor(np.random.randn(100000)).cuda()
%timeit x.dot(y)

257 µs ± 7.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
# ================================================================== #
#                         4. Input pipeline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass


torch.Size([3, 32, 32])
6


In [38]:
x = torch.randn(100, 5)
y = torch.randn(100, 1)
dataset = torch.utils.data.TensorDataset(x, y)
loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=10, 
                                           shuffle=False)

In [39]:
for x_batch, y_batch in loader:
    print(x_batch)
    print(y_batch)
    break

tensor([[ 0.1084, -0.2851,  1.2174, -0.8970,  0.7005],
        [-1.0344,  1.0966,  0.0292,  0.2165,  0.6746],
        [ 0.6224,  0.7937, -0.9416,  0.1784, -0.4309],
        [ 1.0065,  0.1071, -0.3768,  0.7618, -0.0308],
        [-0.3101, -0.5885,  1.0506, -1.2406,  0.8871],
        [-0.9963,  1.0351,  2.1234,  0.5735, -0.0910],
        [ 1.1783,  0.1793,  0.1165,  0.7647, -0.0882],
        [-0.5113,  0.3593, -0.4507,  0.7896, -1.9649],
        [-0.1998, -0.3947, -0.3703, -2.2381,  0.2017],
        [-2.3404, -0.3899,  0.1733,  2.2053, -0.6725]])
tensor([[-1.1190],
        [ 0.7378],
        [ 0.0999],
        [-1.1138],
        [-0.1022],
        [-0.8227],
        [-0.7476],
        [ 1.4706],
        [-0.0587],
        [ 1.4130]])


In [40]:
dataset[0], x[0], y[0]

((tensor([ 0.1084, -0.2851,  1.2174, -0.8970,  0.7005]), tensor([-1.1190])),
 tensor([ 0.1084, -0.2851,  1.2174, -0.8970,  0.7005]),
 tensor([-1.1190]))

In [46]:
# ================================================================== #
#                5. Input pipeline for custom dataset                 #
# ================================================================== #
import os
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, folder_name):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        self.folder_name = folder_name
        self.files = os.listdir(self.folder_name)
        
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        return self.files[index]
    
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.files)

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset(folder_name='.')
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=4, 
                                           shuffle=True)


In [47]:
custom_dataset[1]

'.ipynb_checkpoints'

In [48]:
for names in train_loader:
    print(names)
    break

['4. Feed forward neural network.ipynb', 'model.ckpt', 'pytorch-tutorial-master.zip', 'science.pdf']


In [28]:
a = [1,2,3]
len(a), a.__len__()

(3, 3)

In [32]:
dir(a)
a[1], a.__getitem__(1)

(2, 2)

In [ ]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

In [ ]:
# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))